In [ ]:
###
# This code is for training YOLOv8. Please specify GPU, data directory, YAML files, 
# and other parameters according to your actual environment.
###

In [ ]:
from __future__ import print_function

import glob, os, random, torch, timm, shutil, pickle, time, yaml, json, gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl

from PIL import Image, ImageEnhance, ImageOps, ImageDraw
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from pathlib import Path
from pprint import pprint
from tempfile import TemporaryDirectory
from ultralytics import YOLO

os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

seed = 0
Image.MAX_IMAGE_PIXELS = None
%matplotlib inline

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [ ]:
##########

In [ ]:
# Global settings
DEVICE = 0
EPOCH = 500
BATCH = 64
SEED = 0

In [ ]:
def train_model(yaml_file, run_name, seed, device, epochs, batch_size, weights):
    torch.cuda.empty_cache()
    gc.collect()
    
    seed_everything(seed)
    
    model = YOLO(weights)
    
    model.train(
        data=yaml_file,
        epochs=epochs,
        batch=batch_size,
        cache=False,
        device=device,
        project='PAS_glo',
        name=run_name,
        pretrained=True,
        verbose=True,
        workers=24,
        imgsz=640,
        hsv_h=0.2,
        shear=0.1,
        degrees=90.0,
        flipud=0.5,
        fliplr=0.5,
        box=10.0,
        patience=50
    )
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    del model
    gc.collect()

In [ ]:
WEIGHTS = "/PATH/T/YOUR/WEIGHT/FILE.pt"  

# List of configurations for each training run
configs = [
    {"yaml": "YOUR_YAML.yaml", "name": "YOUR_SAVE_NAME"}
]

# 学習実行
for config in configs:
    print(f"Starting training for {config['name']}")
    train_model(config['yaml'], config['name'], SEED, DEVICE, EPOCH, BATCH, WEIGHTS)
    print(f"Completed training for {config['name']}")
